In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [5]:
ids = []
m1 = {
    "query":{
        "match": {"place": 5}
    }
}
R1 = es.search(index='test', body=m1)
ids.append(R1['hits']['hits'][0]['_id'])

m2 = {
    "doc" : {"place": 10}
}
R2 = es.update(index='test', doc_type='t', id='{}'.format(ids[0]),body=m2)

In [7]:
R2['result']

'updated'

In [ ]:
def reset(ind):
    doc = {
        "index": {
        "blocks": {
        "read_only_allow_delete": "false"
        }
        }
    }
    print(es.index(index=ind+'/_settings', doc_type='employee',body=doc))
reset('business')

In [19]:
details = {"Owais":{"Designation":"AI Expert","Salary":50000,"Gender":"Male"},\
           "Pawan":{"Designation":"Java Developer","Salary":35000,"Gender":"Male"},\
          "Ubaid":{"Designation":"IOS Developer","Salary":45000,"Gender":"Male"},\
          "Asif":{"Designation":"Team Lead","Salary":70000,"Gender":"Male"}}
details

{'Asif': {'Designation': 'Team Lead', 'Gender': 'Male', 'Salary': 70000},
 'Owais': {'Designation': 'AI Expert', 'Gender': 'Male', 'Salary': 50000},
 'Pawan': {'Designation': 'Java Developer', 'Gender': 'Male', 'Salary': 35000},
 'Ubaid': {'Designation': 'IOS Developer', 'Gender': 'Male', 'Salary': 45000}}

In [34]:
for each in details.items():
    print(each[0])

Owais
Pawan
Ubaid
Asif


In [46]:
i = 1
es.indices.create(index='office', ignore=400)
for employee in details.items():
    Q = es.index(index='office', doc_type='employee', id=i, body={"Name" : "{}".format(employee[0]) 
                                                                    ,"Designation": "{}".format(employee[1]['Designation']),
                                                                    "Salary": employee[1]['Salary'],
                                                                    "Gender": "{}".format(employee[1]['Gender'])})
    i+=1

In [47]:
print(Q)

{'_index': 'office', '_type': 'employee', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


In [48]:
ids = [1,2,3,4]
Q1 = []
for eachId in ids:
    Q1.append(es.get(index='office', doc_type='employee', id=eachId))

In [52]:
for each in Q1:
    print(each['_id'], each['_source'])

1 {'Name': 'Owais', 'Designation': 'AI Expert', 'Salary': 50000, 'Gender': 'Male'}
2 {'Name': 'Pawan', 'Designation': 'Java Developer', 'Salary': 35000, 'Gender': 'Male'}
3 {'Name': 'Ubaid', 'Designation': 'IOS Developer', 'Salary': 45000, 'Gender': 'Male'}
4 {'Name': 'Asif', 'Designation': 'Team Lead', 'Salary': 70000, 'Gender': 'Male'}


In [54]:
Q2 = es.search(index="office", body={"query": {"match_all": {}}})
Q2['hits']['hits']

[{'_id': '2',
  '_index': 'office',
  '_score': 1.0,
  '_source': {'Designation': 'Java Developer',
   'Gender': 'Male',
   'Name': 'Pawan',
   'Salary': 35000},
  '_type': 'employee'},
 {'_id': '4',
  '_index': 'office',
  '_score': 1.0,
  '_source': {'Designation': 'Team Lead',
   'Gender': 'Male',
   'Name': 'Asif',
   'Salary': 70000},
  '_type': 'employee'},
 {'_id': '1',
  '_index': 'office',
  '_score': 1.0,
  '_source': {'Designation': 'AI Expert',
   'Gender': 'Male',
   'Name': 'Owais',
   'Salary': 50000},
  '_type': 'employee'},
 {'_id': '3',
  '_index': 'office',
  '_score': 1.0,
  '_source': {'Designation': 'IOS Developer',
   'Gender': 'Male',
   'Name': 'Ubaid',
   'Salary': 45000},
  '_type': 'employee'}]

In [56]:
Q3 = es.search(index="office", body={"query": {"match": { "Name":"Owais" }}})
Q3['hits']['hits']

[{'_id': '1',
  '_index': 'office',
  '_score': 0.2876821,
  '_source': {'Designation': 'AI Expert',
   'Gender': 'Male',
   'Name': 'Owais',
   'Salary': 50000},
  '_type': 'employee'}]

In [71]:
result = els.search(index='office', body={"query":{"match":{"Name":"{}".format('Owais')}}})
result['hits']['hits'][0]['_source']['Salary']

50000

In [100]:
es.indices.create(index='new1')

{'acknowledged': True, 'index': 'new1', 'shards_acknowledged': True}

# Creating DB with Sample Data.

In [57]:
import json
from datetime import datetime

In [58]:
from elasticsearch import Elasticsearch
els = Elasticsearch()

In [59]:
timeframe = 'Data'
sql_transaction = []

In [62]:
def create_table():    
    Q1 = els.indices.create(index='sampledata')
    return Q1
print(create_table())

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'sampledata'}


In [63]:
def format_data(data):
    data = data.replace('\n',' newlinechar ').replace('\r',' newlinechar ').replace('"',"'")
    return data

In [96]:
def find_parent(pid):
    try:
        #sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        result = els.search(index='sampledata', body={"query":{"match":{"comment_id":"{}".format(pid)}}})
        if result['hits']['hits'] != []:
            result = result['hits']['hits'][0]['_source']['comment']
            print("find_parent Function: ", result)
            return result
        else: return False
    except Exception as e:
        print(str(e))
        return False

In [98]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        result = els.search(index='sampledata',body={"query":{"match":{"parent_id":"{}".format(pid)}}})
        if result ['hits']['hits'] != []:
            result = result['hits']['hits'][0]['_source']['score']
            print("find_existing_score Function: ", result)
            return result
        else: return False
    except Exception as e:
        print(str(e))
        return False

In [99]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]':
        return False
    elif data == '[removed]':
        return False
    else:
        return True

In [ ]:
def sql_insert_replace_comment(commentid,parentid,parent,comment,subreddit,time,score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, parent = ?, comment = ?, subreddit = ?, unix = ?, score = ? WHERE parent_id =?;""".format(parentid, commentid, parent, comment, subreddit, int(time), score, parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('sql_insert_replace_comment Function',str(e))

In [ ]:
row_counter = 0
paired_rows = 0
with open('Data',encoding='utf8', errors='ignore')as f:
    for row in f:
        row_counter += 1
        row = json.loads(row)
        parent_id = row['parent_id']
        body = format_data(row['body'])
        created_utc = row['created_utc']
        score = row['score']
        comment_id = row['name']
        subreddit = row['subreddit']
        parent_data = find_parent(parent_id)
        if score >= 2:
            existing_comment_score = find_existing_score(parent_id)
            if existing_comment_score:
                if score > existing_comment_score:
                    if acceptable(body):
                        sql_insert_replace_comment(comment_id,parent_id,parent_data,body,subreddit,created_utc,score)
